In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
dataset = pd.read_csv('/home/urwa/Documents/side_projects/urban/data/featureData/lga.csv')

In [4]:
dataset.shape

(8757, 1045)

In [5]:
dataset.head(3)

,Date,Hour,1,10,100,101,102,106,107,108,...,91_lag_3,92_lag_3,93_lag_3,94_lag_3,95_lag_3,96_lag_3,97_lag_3,98_lag_3,99_lag_3,arrival_lag_3
0,2018-01-01,3,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0
1,2018-01-01,4,1,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2018-01-01,5,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [6]:
lag_columns = [c for c in dataset.columns if 'lag' in c]
len(lag_columns)

774

In [7]:
dataset = dataset[[c for c in dataset.columns if c not in lag_columns]]
dataset.shape

(8757, 271)

In [8]:
DateColumns = ['Date']

ext_columns = ['Dow', 'arrival','maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow']

targetColumns = [c for c in dataset.columns if c not in ext_columns and \
                c not in DateColumns and c not in lag_columns and c != 'Hour']
len(targetColumns)

257

In [9]:
features_cols = [c for c in dataset.columns if c not in targetColumns and c not in DateColumns]
len(features_cols)

13

In [10]:
sep = int(0.75*len(dataset))
print(sep)


trainData = dataset[:sep]
testData = dataset[sep:]

print(trainData.shape)
print(testData.shape)

6567
(6567, 271)
(2190, 271)


In [11]:
X_train = trainData[features_cols].values
X_train = torch.tensor(X_train).float().to(device)
print(X_train.shape)

y_train = trainData[targetColumns].values
y_train = torch.tensor(y_train).float().to(device)
print(y_train.shape)

X_test = testData[features_cols].values
X_test = torch.tensor(X_test).float().to(device)
print(X_test.shape)

y_test = testData[targetColumns].values
y_test = torch.tensor(y_test).float().to(device)
print(y_test.shape)

torch.Size([6567, 13])
torch.Size([6567, 257])
torch.Size([2190, 13])
torch.Size([2190, 257])


In [12]:
def create_inout_sequences(x,y, tw):
    inout_seq = []
    L = len(x)
    for i in range(L-tw):
        train_seq_x = x[i:i+tw]
        train_seq_y = y[i:i+tw]
#         train_seq = torch.cat((train_seq_x,train_seq_y),axis=1)
        
#         train_label = y[i+tw:i+tw+1]
        train_label = y[i+1:i+tw+1]
        inout_seq.append((train_seq_x, train_seq_y ,train_label))
    return inout_seq

In [13]:
bptt = 6

In [14]:
train_inout_seq = create_inout_sequences(X_train,y_train, bptt)

In [15]:
train_inout_seq[0][0].shape,train_inout_seq[0][1].shape, train_inout_seq[0][2].shape

(torch.Size([6, 13]), torch.Size([6, 257]), torch.Size([6, 257]))

In [16]:
test_inout_seq = create_inout_sequences(X_test,y_test, bptt)

In [17]:
class LSTM(nn.Module):
    def __init__(self, feat_size=1, hidden_layer_size=100, network_size=1, layers=1, communities=10):
        super().__init__()
        
        # aggregation
        self.attachment_matrix = torch.nn.Parameter(torch.randn(network_size,communities))
        self.attachment_matrix.requires_grad = True
        
        
        self.hidden_layer_size = hidden_layer_size
        
        self.hidden_cell = (torch.zeros(layers,1,self.hidden_layer_size),
                    torch.zeros(layers,1,self.hidden_layer_size))
        
        lstm_input = communities + feat_size
        self.lstm = nn.LSTM(input_size=lstm_input, hidden_size=hidden_layer_size, num_layers=layers)

        #disaggregation
#         self.linear_1 = nn.Linear(hidden_layer_size, hidden_layer_size)
        self.linear_2 = nn.Linear(hidden_layer_size, network_size)


    def forward(self, input_seq, feat):
        
        w = F.softmax(self.attachment_matrix, dim=1)
        x = torch.matmul(input_seq, self.attachment_matrix)
        x = torch.cat((x,feat),axis=1)

        
        lstm_out, self.hidden_cell = self.lstm(x.view(len(input_seq) ,1, -1), self.hidden_cell)
        
        predictions = self.linear_2(lstm_out.view(len(input_seq), -1))
#         predictions = F.relu(predictions)
#         predictions = self.linear_2(predictions)
        
        return predictions

In [18]:
def evaluate(model):
    model.eval()
    prediction = []
    with torch.no_grad():
        for feat,seq, labels in test_inout_seq:
            model.hidden = (torch.zeros(layers, 1, model.hidden_layer_size),
                            torch.zeros(layers, 1, model.hidden_layer_size))
            prediction.append(model(seq,feat)[-1])

    y_test_ = torch.stack([labels[-1] for feat,seq, labels in test_inout_seq], axis=0).detach().cpu().numpy()
    y_pred_ = torch.stack(prediction).detach().cpu().numpy()

    r2 = r2_score(y_test_, y_pred_, multioutput='variance_weighted')
    rmse = mean_squared_error(y_test_, y_pred_)
    mae = mean_absolute_error(y_test_, y_pred_)
#     print("r2: ",r2)
    return (r2, rmse, mae)

In [19]:
layers = 1
communities = 24
network_size = len(targetColumns)
feat_size = len(features_cols)

model = LSTM(feat_size = feat_size, hidden_layer_size=communities,
             network_size=network_size, layers=layers,
            communities=communities).to(device)

loss_function = nn.L1Loss()   
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

In [20]:
epochs = 100


for i in range(epochs):
    model.train()
    for feat,seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(layers, 1, model.hidden_layer_size).to(device),
                        torch.zeros(layers, 1, model.hidden_layer_size).to(device))

        y_pred = model(seq, feat)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()
        
    scheduler.step()
    if i%10 == 1:
        r2, rmse, mae = evaluate(model)
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f} r2: {r2:5.3f} rmse: {rmse:5.3f} mae: {mae:5.3f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

KeyboardInterrupt: 

In [21]:
evaluate(model)

(0.7046127387300352, 6.887028, 1.0707647)

In [26]:
# attachment = torch.argmax(F.softmax(model.attachment_matrix, dim=1), dim=1).detach().cpu().numpy()
# community_assignment = dict(zip(targetColumns, attachment))
# community_assignment

In [27]:
# community_assignment

In [32]:
attachment = model.attachment_matrix.detach().cpu().numpy()

In [33]:
attachment.shape

(257, 24)

In [30]:
len(targetColumns)

257

In [35]:
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score

In [49]:
for i in range(2,24):
    print('========== '+str(i)+' ============')
    kmeans = KMeans(n_clusters=i, random_state=1).fit(attachment)
    labels = kmeans.labels_
    print(davies_bouldin_score(attachment, labels))

========== 2 ============
2.8963139551937362
========== 3 ============
3.2892934667507987
========== 4 ============
3.129459677258672
========== 5 ============
3.0258845288014555
========== 6 ============
3.076842440938373
========== 7 ============
2.9947178511630725
========== 8 ============
2.877054105184271
========== 9 ============
2.8290008475947457
========== 10 ============
2.8404560378365904
========== 11 ============
2.774318371378216
========== 12 ============
2.719678940570788
========== 13 ============
2.547455966446989
========== 14 ============
2.5443252809522585
========== 15 ============
2.5685699026103843
========== 16 ============
2.577130689691856
========== 17 ============
2.4756219256433702
========== 18 ============
2.404953712633187
========== 19 ============
2.555520310984024
========== 20 ============
2.417650742023066
========== 21 ============
2.4885713148860455
========== 22 ============
2.427904609154537
========== 23 ============
2.442913565151824


In [50]:
labels = kmeans.labels_

In [51]:
labels

array([ 6,  8, 22,  6,  7,  2,  6,  4,  6, 10,  0, 22,  3,  8,  8,  2,  4,
        2, 19, 12,  2, 19,  1, 16, 18, 13, 18, 14, 19, 19, 17,  6,  8,  6,
       10,  5, 14,  7,  3,  5,  4, 10,  2, 10, 10,  7,  4, 18, 16, 13, 17,
        6,  4,  2,  5, 11, 12, 12, 11,  7, 20, 21, 21, 10, 13,  3, 20,  7,
       22,  7, 19,  8,  4,  5, 10,  0, 20, 22,  2,  2, 19,  8, 19, 10,  8,
        7, 22, 20,  5, 12, 13,  7, 12, 14, 10,  8, 12, 21,  1,  7, 13,  5,
        2, 10,  7, 19,  6, 17, 10, 13, 22, 12, 15, 12, 22,  4,  7, 22, 22,
       13, 22,  7, 17, 17, 18,  3,  2,  8, 22,  3, 10,  0, 19,  7,  3, 21,
       19, 16,  1, 18, 18,  1, 18, 13, 13, 13, 19, 10, 22, 22, 10, 21, 13,
        5, 17,  3,  5,  1, 12, 14,  5, 10, 22,  8, 15,  1,  5, 18,  3,  4,
       12,  9,  3, 16, 22,  1, 10,  3,  5,  7, 21, 19,  3,  9, 13,  6, 13,
        5,  7,  7,  6, 18, 19, 10,  7,  1, 13, 11, 13,  1, 18,  8, 20,  6,
       12, 14, 10,  2,  6,  7,  2,  2,  5, 20, 18, 20, 19,  4, 22, 12, 13,
       20, 22, 13,  2,  2

In [52]:
community_assignment = dict(zip(targetColumns, labels.astype(str)))

In [53]:
import json

with open('lga_single.json', 'w') as fp:
    json.dump(community_assignment, fp)

In [24]:
# 20 comm
# 0.505

In [25]:
# 50 comm
# 